In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers
import ast

In [32]:
df = pd.read_csv('prepared_data/back.csv')
df

,Unnamed: 0,Category,Accelerometer,Gyroscope,Magnetometer,Duration
0,0,CLIMBING,"[[-6.850950241088867, 6.60105037689209, -1.287...","[[0.29122498631477356, 0.5505499839782715, 0.2...","[[65.8125, -19.012500762939453, 16.80000114440...",20
1,1,FALLING,"[[-5.451000213623047, 7.82895040512085, 1.9920...","[[-0.07741250097751617, 0.2190375030040741, -0...","[[32.34375, -38.94375228881836, -29.3437519073...",70
2,2,SUCCESSFULL,"[[6.903000354766846, -1.7880001068115234, -4.9...","[[-0.23718750476837158, 0.36121249198913574, 0...","[[-37.91250228881836, 10.143750190734863, -6.3...",85
3,3,WALKING,"[[-6.942000389099121, 5.0400004386901855, -0.0...","[[0.1816374957561493, -0.605412483215332, 0.25...","[[34.14375305175781, -34.20000076293945, 15.95...",25
4,4,CLIMBING,"[[-6.130050182342529, 6.98805046081543, 2.9569...","[[0.11261250078678131, 0.16541250050067902, -0...","[[30.187501907348633, -6.056250095367432, -4.1...",50
5,5,FALLING,"[[-5.57295036315918, 7.07205057144165, 1.01295...","[[0.46131250262260437, 0.5150749683380127, -0....","[[32.34375, -40.38750076293945, -23.2875003814...",2
6,6,SUCCESSFULL,"[[6.709050178527832, -7.123950481414795, -0.58...","[[0.08277499675750732, -0.210099995136261, -0....","[[-26.45625114440918, 22.087501525878906, -27....",85
7,7,WALKING,"[[-3.0010502338409424, 7.975050449371338, 0.62...","[[-0.16554999351501465, -1.0306999683380127, -...","[[21.11250114440918, -35.75625228881836, 12.90...",25
8,8,CLIMBING,"[[-7.414050579071045, 7.0819501876831055, 1.23...","[[0.489362508058548, -0.05211250111460686, -0....","[[32.04375076293945, -19.743751525878906, -18....",60
9,9,FALLING,"[[-4.780050277709961, 6.909000396728516, 0.280...","[[-0.20446249842643738, 0.1387374997138977, 0....","[[37.80000305175781, -22.856250762939453, -12....",2


In [27]:
# Mask all gyro values if the label is Falling
for row in df.values:
    if(row[1] == "FALLING"):
        row[3] = np.zeros((len(row[3]),3))

# Mask all magnetometer values if the label is Walking
for row in df.values:
    if(row[1] == "WALKING"):
        row[3] = np.zeros((len(row[3]),3))

In [28]:
# Convert string to list of list of floats
df['Accelerometer'] = df['Accelerometer'].apply(eval)
df['Gyroscope'] = df['Gyroscope'].apply(eval)
df['Magnetometer'] = df['Magnetometer'].apply(eval)

# Find the maximum length among the sequences
max_length = max(
    df['Accelerometer'].apply(len).max(),
    df['Gyroscope'].apply(len).max(),
    df['Magnetometer'].apply(len).max()
)

In [26]:
# Create empty arrays to store the masked sequences
X_accelerometer = np.zeros((len(df), max_length, 3))
X_gyroscope = np.zeros((len(df), max_length, 3))
X_magnetometer = np.zeros((len(df), max_length, 3))

# Mask the sequences with zeros
for i, (acc, gyr, mag) in enumerate(zip(df['Accelerometer'], df['Gyroscope'], df['Magnetometer'])):
    X_accelerometer[i, :len(acc)] = acc
    X_gyroscope[i, :len(gyr)] = gyr
    X_magnetometer[i, :len(mag)] = mag


# Split data into features (X) and labels (y)
y = df['Category'].values

# Encode labels as numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_accelerometer, y, test_size=0.2, random_state=42)

# Create the LSTM model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(3,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=1)

ValueError: could not convert string to float: '[[-6.850950241088867, 6.60105037689209, -1.287000060081482], [-6.942000389099121, 6.268050193786621, -1.0090500116348267], [-7.323000431060791, 5.983050346374512, -0.5689499974250793], [-7.216950416564941, 6.000000476837158, -0.046950001269578934], [-7.030950546264648, 6.628950119018555, -0.0970500037074089], [-7.699050426483154, 6.628950119018555, -1.7149500846862793], [-7.656000137329102, 5.533050060272217, -1.540950059890747], [-6.672000408172607, 5.64300012588501, -0.42195001244544983], [-6.727050304412842, 6.150000095367432, -0.6480000019073486], [-9.001049995422363, 5.692950248718262, 0.20595000684261322], [-11.296950340270996, 6.319950103759766, 0.4740000367164612], [-10.797000885009766, 7.7500505447387695, -1.2220500707626343], [-9.40095043182373, 8.209050178527832, -2.095950126647949], [-9.1399507522583, 8.176050186157227, -1.828050136566162], [-8.224950790405273, 8.89905071258545, -2.0220000743865967], [-7.204950332641602, 8.254

In [22]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_train, y_train)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)
model.evaluate

1/1 [==============================] - 2s 2s/step - loss: -18.9033 - accuracy: 0.4400
Test Loss: -18.90326499938965
Test Accuracy: 0.4399999976158142


<bound method Model.evaluate of <keras.engine.sequential.Sequential object at 0x1399d2e50>>